### Notebook for gathering the dataset of the s&p500 from wikipedia.

I am going to collect daily the data of the companies belonging to th s&p500

In [3]:
import pandas as pd


In [4]:

companies_df, changes_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")

In [5]:
companies_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [6]:
changes_df.head()

Date  Added                 Removed  \
                Date Ticker        Security  Ticker   
0    January 5, 2023    NaN             NaN     VNO   
1    January 4, 2023   GEHC   GE HealthCare     NaN   
2  December 22, 2022   STLD  Steel Dynamics    ABMD   
3  December 19, 2022   FSLR     First Solar    FBHS   
4  December 19, 2022    NaN             NaN     MBC   

                                   \
                         Security   
0            Vornado Realty Trust   
1                             NaN   
2                         Abiomed   
3  Fortune Brands Home & Security   
4                     MasterBrand   

                                              Reason  
                                              Reason  
0                   Market capitalization change.[4]  
1  S&P 500 constituent General Electric spun off ...  
2  S&P 500 constituent Johnson & Johnson acquired...  
3                   Market capitalization change.[7]  
4                   Market capitalization change.[7]

The list of changes made are only since 1997. Therefore i do not think we can completely trust this historical data

In [7]:
#all the companies that formed part in the sp500 index (wikipedia)
companies = companies_df['Symbol'].tolist()
print("companies currently listed in the sp500 index: ", len(companies))

temp = changes_df.Added.Ticker.to_list()
temp += changes_df.Removed.Ticker.to_list()


#all the companies that formed part in the sp500 index (wikipedia)
all_companies = companies + temp

# delete duplicates
all_companies = list(dict.fromkeys(all_companies))
print("all companies that formed part in the sp500 index: ", len(all_companies))

companies currently listed in the sp500 index:  503
all companies that formed part in the sp500 index:  807


In [8]:
for i, company in enumerate(all_companies):
    if not isinstance(company, str):
        print(company) 
        print(i)

nan
503


In [9]:
# January 5, 2023 to yyyy-mm-dd
months = {"january": "01", "february": "02", "march": "03", "april": "04", "may": "05", "june": "06",
    "july": "07", "august": "08", "september": "09", "october": "10", "november": "11", "december": "12"}

import re
def convert_date(date):
    date = date.lower()
    # take out month, day, year
    month, day, year = re.findall(r"\w+", date)

    # convert month to number
    if len(month) == 1:
        month = "0" + month
    if len(day) == 1:
        day = "0" + day
    
    return f"{year}-{months[month]}-{day}"

In [10]:
# create a dictionary with the companies and more data

counter_missing_removed = 0
counter_missing_added = 0
companies_dict = {}
exclude = ["FB", "WL"]
ticker_names_dict = {
    "CMCSK": "CMCSA",
    "ENDP": "ENDPQ",
    
}
for company in all_companies:
    if company in exclude:
        continue
    if company in ticker_names_dict:
        company = ticker_names_dict[company]
    if company in companies:
        companies_dict[company] = {
            'name': companies_df[companies_df['Symbol'] == company]['Security'].values[0],
            'sector': companies_df[companies_df['Symbol'] == company]['GICS Sector'].values[0],
            'industry': companies_df[companies_df['Symbol'] == company]['GICS Sub-Industry'].values[0],
            'date_added': companies_df[companies_df['Symbol'] == company]['Date added'].values[0],
            'date_removed': None,
            'reason_removed': None,
            'is_in_index': True
        }
    else:
        try:
            try:
                name = changes_df.Removed[changes_df.Removed['Ticker'] == company]['Security'].values[0]
            except:
                name = changes_df.Added[changes_df.Added['Ticker'] == company]['Security'].values[0]

            # standardaize date format to YYYY-MM-DD
            
            companies_dict[company] = {
                'name': name,
                'sector': None,
                'industry': None,
                'date_added': None,
                'date_removed': None,
                'reason_removed': None,
                'is_in_index': False
            }
            try:
                date =  changes_df[changes_df['Removed']["Ticker"] == company]['Date'].values[0][0]
                companies_dict[company]["date_removed"] = convert_date(date)
                companies_dict[company]["reason_removed"] = changes_df[changes_df['Removed']["Ticker"] == company]['Reason'].values[0][0]
            except:
                counter_missing_removed += 1
                print(f"could not find date removed for {company}: current count: {counter_missing_removed}")
                
            try:
                date = changes_df[changes_df['Added']["Ticker"] == company]['Date'].values[0][0]
                companies_dict[company]["date_added"] = convert_date(date)
            except:
                counter_missing_added += 1
                print(f"could not find date added for {company} current count: {counter_missing_added}")
        except Exception as e:
            print(f"could not find company {company} in changes_df")
            print(e)
            

could not find company nan in changes_df
index 0 is out of bounds for axis 0 with size 0
could not find date removed for WLTW: current count: 1
could not find company ENDPQ in changes_df
index 0 is out of bounds for axis 0 with size 0
could not find date removed for DISCK: current count: 2
could not find date removed for KORS: current count: 3
could not find date removed for DLPH: current count: 4
could not find date removed for JOYG: current count: 5
could not find date removed for PCLN: current count: 6
could not find date removed for HRS: current count: 7
could not find date removed for COG: current count: 8
could not find date removed for JEC: current count: 9
could not find date removed for TSO: current count: 10
could not find date removed for LUK: current count: 11
could not find date removed for KFT: current count: 12
could not find date removed for WLP: current count: 13
could not find date removed for FSR: current count: 14
could not find date added for VNO current count: 1
c

In [11]:
changes_df[changes_df['Removed']["Ticker"] == "LDW"]

Date  Added          Removed           \
                  Date Ticker Security  Ticker Security   
314   December 7, 1999   YHOO   Yahoo!     LDW  Laidlaw   
316  December 11, 1998    FSR  Firstar     LDW    Amoco   

                                      Reason  
                                      Reason  
314       Market capitalization change.[240]  
316  British Petroleum purchased Amoco.[242]

In [12]:
all_companies_df = pd.DataFrame.from_dict(companies_dict, orient='index')
all_companies_df.reset_index(names=["ticker"], inplace=True)
all_companies_df.head()

,ticker,name,sector,industry,date_added,date_removed,reason_removed,is_in_index
0,MMM,3M,Industrials,Industrial Conglomerates,1957-03-04,None,None,True
1,AOS,A. O. Smith,Industrials,Building Products,2017-07-26,None,None,True
2,ABT,Abbott,Health Care,Health Care Equipment,1957-03-04,None,None,True
3,ABBV,AbbVie,Health Care,Pharmaceuticals,2012-12-31,None,None,True
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,2011-07-06,None,None,True


In [13]:
len(all_companies_df)

803

In [14]:
all_companies_df.to_csv("sp500/all_companies.csv")

In [15]:
import yfinance as yf

# get historical data for apple
data = yf.download("META", period="max")

[*********************100%***********************]  1 of 1 completed


In [16]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-05-18,42.049999,45.000000,38.000000,38.230000,38.230000,573576400
2012-05-21,36.529999,36.660000,33.000000,34.029999,34.029999,168192700
2012-05-22,32.610001,33.590000,30.940001,31.000000,31.000000,101786600
2012-05-23,31.370001,32.500000,31.360001,32.000000,32.000000,73600000
2012-05-24,32.950001,33.209999,31.770000,33.029999,33.029999,50237200
...,...,...,...,...,...,...
2023-03-06,188.000000,189.660004,184.639999,184.899994,184.899994,33209400
2023-03-07,189.000000,190.360001,184.009995,184.509995,184.509995,36701500
2023-03-08,182.869995,185.259995,181.339996,184.970001,184.970001,19432400


In [17]:
!mkdir sp500

mkdir: cannot create directory ‘sp500’: File exists


In [ ]:
!mkdir sp500/data

In [18]:
import os
errors = 0

companies_error = []
names_error = []

found_data = []
for i, (company, name) in enumerate(zip(all_companies_df['ticker'], all_companies_df['name'])):
    # check if data already exists
    if os.path.exists(f"data/{company}.csv"):
        continue
    try:
        data = yf.download(company, start="1985-01-01")

        if len(data) == 0:
            data = yf.download(company.replace(".", "-"), start="1985-01-01")
        

        if len(data) > 0:
            data.to_csv(f"sp500/data/{company}.csv")
            found_data.append(True)
        else:
            errors += 1
            companies_error.append(company)
            names_error.append(name)
            found_data.append(False)
            print(f"could not find company {company}: {name} in yfinance")
    except:
        errors += 1
        companies_error.append(company)
        names_error.append(name)
        found_data.append(False)
        print(f"could not find company {company}: {name} in yfinance")
    

[*********************100%***********************]  1 of 1 completed
could not find company MMM: 3M in yfinance
[*********************100%***********************]  1 of 1 completed
could not find company AOS: A. O. Smith in yfinance
[*********************100%***********************]  1 of 1 completed
could not find company ABT: Abbott in yfinance
[*********************100%***********************]  1 of 1 completed
could not find company ABBV: AbbVie in yfinance
[*********************100%***********************]  1 of 1 completed
could not find company ACN: Accenture in yfinance
[*********************100%***********************]  1 of 1 completed
could not find company ATVI: Activision Blizzard in yfinance
[*********************100%***********************]  1 of 1 completed
could not find company ADM: ADM in yfinance
[*********************100%***********************]  1 of 1 completed
could not find company ADBE: Adobe Inc. in yfinance
[*********************100%***********************] 

In [ ]:
# write a txt file with the companies that could not be found
with open("sp500/companies_not_found.txt", "w") as f:
    for company, name in zip(companies_error, names_error):
        f.write(f"{company}: {name}\n")

In [ ]:
# check if any of the tickers belong to the sp500
for company in companies_error:
    if sum(all_companies_df[all_companies_df['ticker'] == company].is_in_index) > 0:
        print(company)